In [1]:
from typing import Callable, Literal

import numpy as np
import polars as pl
from polars.testing import (assert_frame_equal, assert_series_equal,
                            assert_series_not_equal)


## pl.Series

In [2]:
s1 = pl.Series("s", [1, 2, 3])
# s1 = pl.Series("s", np.array([1, 2, 3]))
print(s1)

shape: (3,)
Series: 's' [i64]
[
	1
	2
	3
]


In [3]:
s2 = pl.Series("s", [1, 2, 3], dtype=pl.Int64)
print(s2)

shape: (3,)
Series: 's' [i64]
[
	1
	2
	3
]


In [4]:
assert_series_equal(s1, s2)

In [5]:
s3 = pl.Series("s", [1, 2, 3], dtype=pl.Float64)
print(s3)

shape: (3,)
Series: 's' [f64]
[
	1.0
	2.0
	3.0
]


In [6]:
s4 = pl.Series([1, 2, 3], dtype=pl.Float64)
print(s4)

shape: (3,)
Series: '' [f64]
[
	1.0
	2.0
	3.0
]


In [7]:
assert_series_not_equal(s3, s4)  # name mismatch ("s" vs. "")

In [8]:
s5 = pl.Series([1, None, "3"], strict=False)
print(s5)

shape: (3,)
Series: '' [str]
[
	"1"
	null
	"3"
]


### Attributes and methods

In [9]:
print(s1.shape)

(3,)


In [10]:
print(s1.item(0))

1


In [11]:
print(s1.item(-1))

3


In [12]:
print(s1.top_k(2))

shape: (2,)
Series: 's' [i64]
[
	3
	2
]


In [13]:
print(s1.top_k(2).sort())

shape: (2,)
Series: 's' [i64]
[
	2
	3
]


In [14]:
s6 = pl.Series([5, 0, 6])  # [True, False, True]
print(s1.zip_with(s1 < s6, s6))

shape: (3,)
Series: 's' [i64]
[
	1
	0
	3
]


In [15]:
print(s1.zip_with(pl.Series([True, False, True]), s6))

shape: (3,)
Series: 's' [i64]
[
	1
	0
	3
]


## pl.DataFrame

In [16]:
df1 = pl.DataFrame({"col1": [1, 2, 3], "col2": ["x", "y", "z"]})
print(df1)

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ str  │
╞══════╪══════╡
│ 1    ┆ x    │
│ 2    ┆ y    │
│ 3    ┆ z    │
└──────┴──────┘


In [17]:
df2 = pl.DataFrame(
    {"col1": [1, 2, 3], "col2": ["x", "y", "z"]},
    schema={"col1": pl.Int64, "col2": pl.String},
)
print(df2)

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ str  │
╞══════╪══════╡
│ 1    ┆ x    │
│ 2    ┆ y    │
│ 3    ┆ z    │
└──────┴──────┘


In [18]:
assert_frame_equal(df1, df2)

In [19]:
# ValueError: the given column-schema names do not match the data dictionary
# pl.DataFrame({"col1": [1, 2, 3], "col2": ["x", "y", "z"]}, schema={"col1": pl.Int64})


In [20]:
df3 = pl.DataFrame(
    {"col1": [1, 2, 3], "col2": ["x", "y", "z"]},
    schema_overrides={"col1": pl.Int64},
)
print(df3)

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ str  │
╞══════╪══════╡
│ 1    ┆ x    │
│ 2    ┆ y    │
│ 3    ┆ z    │
└──────┴──────┘


In [21]:
assert_frame_equal(df1, df3)

In [22]:
df4 = pl.DataFrame([[1, 2, 3], ["x", "y", "z"]], schema=["col1", "col2"])
print(df4)

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ str  │
╞══════╪══════╡
│ 1    ┆ x    │
│ 2    ┆ y    │
│ 3    ┆ z    │
└──────┴──────┘


In [23]:
df5 = pl.from_dict({"col1": [1, 2, 3], "col2": ["x", "y", "z"]})
print(df5)

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ str  │
╞══════╪══════╡
│ 1    ┆ x    │
│ 2    ┆ y    │
│ 3    ┆ z    │
└──────┴──────┘


In [24]:
assert_frame_equal(df1, df5)

In [25]:
df6 = pl.from_repr(
    """
    shape: (3, 2)
    ┌──────┬──────┐
    │ col1 ┆ col2 │
    │ ---  ┆ ---  │
    │ i64  ┆ str  │
    ╞══════╪══════╡
    │ 1    ┆ x    │
    │ 2    ┆ y    │
    │ 3    ┆ z    │
    └──────┴──────┘
    """
)
print(df6)

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ str  │
╞══════╪══════╡
│ 1    ┆ x    │
│ 2    ┆ y    │
│ 3    ┆ z    │
└──────┴──────┘


In [26]:
assert_frame_equal(df1, df6)

### Attributes and methods

In [27]:
df1.shape

(3, 2)

In [28]:
print(f"{df1.height=}\n{df1.width=}")

df1.height=3
df1.width=2


In [29]:
print(df1.columns)

['col1', 'col2']


In [30]:
print(df1.with_row_index())

shape: (3, 3)
┌───────┬──────┬──────┐
│ index ┆ col1 ┆ col2 │
│ ---   ┆ ---  ┆ ---  │
│ u32   ┆ i64  ┆ str  │
╞═══════╪══════╪══════╡
│ 0     ┆ 1    ┆ x    │
│ 1     ┆ 2    ┆ y    │
│ 2     ┆ 3    ┆ z    │
└───────┴──────┴──────┘


In [31]:
print(df1.head())

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ str  │
╞══════╪══════╡
│ 1    ┆ x    │
│ 2    ┆ y    │
│ 3    ┆ z    │
└──────┴──────┘


In [32]:
print(df1.glimpse())

Rows: 3
Columns: 2
$ col1 <i64> 1, 2, 3
$ col2 <str> 'x', 'y', 'z'

None


## iter columns

In [33]:
for ser in df1.iter_columns():
    print(ser, end="\n\n")

shape: (3,)
Series: 'col1' [i64]
[
	1
	2
	3
]

shape: (3,)
Series: 'col2' [str]
[
	"x"
	"y"
	"z"
]



In [34]:
df1.get_columns()

[shape: (3,)
 Series: 'col1' [i64]
 [
 	1
 	2
 	3
 ],
 shape: (3,)
 Series: 'col2' [str]
 [
 	"x"
 	"y"
 	"z"
 ]]

## iter rows

In [35]:
for row in df1.iter_rows():
    print(row)

(1, 'x')
(2, 'y')
(3, 'z')


In [36]:
for row in df1.iter_rows(named=True):
    print(row["col1"], row["col2"])

1 x
2 y
3 z


## Conversions

In [37]:
print(s1.to_frame())

shape: (3, 1)
┌─────┐
│ s   │
│ --- │
│ i64 │
╞═════╡
│ 1   │
│ 2   │
│ 3   │
└─────┘


In [38]:
print(df1.to_series())

shape: (3,)
Series: 'col1' [i64]
[
	1
	2
	3
]


## add pl.Series to pl.DataFrame

In [39]:
print(df1.with_columns(s1))

shape: (3, 3)
┌──────┬──────┬─────┐
│ col1 ┆ col2 ┆ s   │
│ ---  ┆ ---  ┆ --- │
│ i64  ┆ str  ┆ i64 │
╞══════╪══════╪═════╡
│ 1    ┆ x    ┆ 1   │
│ 2    ┆ y    ┆ 2   │
│ 3    ┆ z    ┆ 3   │
└──────┴──────┴─────┘


## pl.concat()

In [40]:
s_v1 = pl.Series("s_v1", [1, 2, 3])
s_v2 = pl.Series("s_v2", [4, 5, 6])
print(pl.concat([s_v1, s_v2]))

shape: (6,)
Series: 's_v1' [i64]
[
	1
	2
	3
	4
	5
	6
]


In [41]:
df_v1 = pl.DataFrame({"col1": [1, 2, 3], "col2": [4, 5, 6]})
df_v2 = pl.DataFrame({"col1": [7, 8, 9], "col2": [10, 11, 12]})
print(pl.concat([df_v1, df_v2], how="vertical"))

shape: (6, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ i64  │
╞══════╪══════╡
│ 1    ┆ 4    │
│ 2    ┆ 5    │
│ 3    ┆ 6    │
│ 7    ┆ 10   │
│ 8    ┆ 11   │
│ 9    ┆ 12   │
└──────┴──────┘


In [42]:
df_h1 = pl.DataFrame({"col1": [1, 2, 3], "col2": [4, 5, 6]})
df_h2 = pl.DataFrame({"col3": [7, 8, 9], "col4": [10, 11, 12]})
print(pl.concat([df_h1, df_h2], how="horizontal"))

shape: (3, 4)
┌──────┬──────┬──────┬──────┐
│ col1 ┆ col2 ┆ col3 ┆ col4 │
│ ---  ┆ ---  ┆ ---  ┆ ---  │
│ i64  ┆ i64  ┆ i64  ┆ i64  │
╞══════╪══════╪══════╪══════╡
│ 1    ┆ 4    ┆ 7    ┆ 10   │
│ 2    ┆ 5    ┆ 8    ┆ 11   │
│ 3    ┆ 6    ┆ 9    ┆ 12   │
└──────┴──────┴──────┴──────┘
